In [109]:
!git config --global user.name "Coen-Molyneaux"
!git config --global user.email "coen.molyneaux@shaw.ca"


In [110]:
!git clone https://github.com/Coen-Molyneaux/ENPH353_Lab2.git


Cloning into 'ENPH353_Lab2'...


In [118]:
%cd /content/ENPH353_Lab2
# remove the nested clone if it exists
!rm -rf ENPH353_Lab2
# start fresh git in THIS folder
!rm -rf .git
!git init
!git config user.name "Your Name"
!git config user.email "you@example.com"
!git branch -M main
!git remote add origin https://<TOKEN>@github.com/Coen-Molyneaux/ENPH353_Lab2.git

!git add .
!git commit -m "Initial commit from Colab"
!git push -u origin main


/content/ENPH353_Lab2
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/ENPH353_Lab2/.git/
/bin/bash: line 1: TOKEN: No such file or directory
On branch main

Initial commit

nothing to commit (create/copy files and use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'origin'


In [117]:
!cd ENPH353_Lab2/
!git init
!git add .
!git commit -m "Lab 2"
!git push

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
^C
On branch master

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	ENPH353_Lab2/
	dotted_line.mp4
	drive/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)
fatal: No configured push destination.
Either specify the URL from the command-line or configure a remote repository using

    git remote add <name> <url>

and then push using the remote name

    git p

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [108]:
import cv2
import numpy
import matplotlib.pyplot as plt

VIDEO_IN = "/content/drive/MyDrive/ENPH_353/LAB_2/raw_video_feed.mp4"
video_capture = cv2.VideoCapture(VIDEO_IN)

default_offset = 50

def grey_scale(frame) -> numpy.ndarray:
  '''
  @brief Grey scales an image
  @param frame The image to be grey scaled
  @return The grey scaled image
  '''
  return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

def blur(frame) -> numpy.ndarray:
  '''
  @brief Blurs an image
  @param frame The image to be blurred
  @return The blurred image
  '''
  return cv2.GaussianBlur(frame, (5, 5), 0)

def threshold(frame, threshold_value: int = 130) -> numpy.ndarray:
  '''
  @brief Thresholds an image
  @param frame The image to be thresholded
  @param threshold_value The threshold value
  @return The thresholded image
  '''
  thresh_frame = cv2.threshold(frame, threshold_value, 255, cv2.THRESH_BINARY)[1]
  return cv2.bitwise_not(thresh_frame)

def lower_frame_band(frame, band_height) -> numpy.ndarray:
  '''
  @brief Gets the lower band of an image
  @param frame The image to be processed
  @param band_height The height of the band
  @return The processed image
  '''
  height = frame.shape[0]
  return frame[height-band_height:height]

def process_img(frame, roi_offset=default_offset) -> numpy.ndarray:
  '''
  @brief Processes an image
  @param frame The image to be processed
  @param roi_offset The offset of the roi
  @return The processed image
  '''
  grey_img = grey_scale(frame)
  blur_img = blur(grey_img)
  thresh_img = threshold(blur_img)
  return lower_frame_band(thresh_img, roi_offset=roi_offset)


def put_dot_on_line(frame: numpy.ndarray, roi: numpy.ndarray, roi_offset: int=default_offset) -> numpy.ndarray:
  '''
  @brief Puts a dot on the line of an image
  @param frame The image to be processed
  @param roi The roi to be processed
  @param roi_offset The offset of the roi
  @return The processed image
  '''
  moments = cv2.moments(roi)
  height = frame.shape[0]
  radius = 15
  filled = -1
  colour = (150,150,0)

  if moments['m00'] != 0:
    cx = int(moments['m10'] / moments['m00'])
    cy = int(moments['m01'] / moments['m00'])
  center = (cx, cy + height - roi_offset)

  cv2.circle(frame, center, radius, colour, filled)
  return frame

def frames_to_video(frames, out_path="dotted_line.mp4", fps=15):
  '''
  @breif Converts a list of frames to a video
  @param frames The list of frames
  @param out_path The path to the output video
  @param fps The frames per second
  '''
  h, w = frames[0].shape[:2]
  writer = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps,(w, h))
  for f in frames:
    writer.write(f)
  writer.release()

In [ ]:
frames_with_dots = []

while True:
  read, frame = video_capture.read()
  if not read:
      break
  img_for_detection = process_img(frame)
  put_dot_on_line(frame, img_for_detection)
  frames_with_dots.append(frame)

if frames_with_dots:
  frames_to_video(frames_with_dots)
  from google.colab import files
  files.download("dotted_line.mp4")
else:
    print("No frames captured. Check if the video path is correct or video_capture is opened.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>